In [30]:
import numpy as np
import matplotlib.pyplot as plt
import heapq

def out_of_bounds(i, j, maze):
    return i < 0 or j < 0 or i >= len(maze) or j >= len(maze[0])

class Node():
    moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    def __init__(self, pos: tuple[int, int]):
        self.pos = pos
        self.visited = False
        self.distance = float('inf')
        self.neighbors: list[Node] = []
        
    def __add__(self, move: tuple[int, int]) -> tuple[int, int]:
        return self.pos[0] + move[0], self.pos[1] + move[1]
    
    def __eq__(self, other):
        # if other is not a node, return false
        if not isinstance(other, Node):
            return False
        return self.pos == other.pos
    
    def __lt__(self, other):
        self.pos < other.pos
    
    def __str__(self):
        return f'{self.pos}'
    
    def __repr__(self):
        return self.__str__()
    
    def __hash__(self):
        return hash(self.pos)

class Maze():
    def __init__(self, maze: np.ndarray[np.ndarray[Node]], start: Node, end: Node, walls: list[tuple[int, int]]):
        self.maze = maze
        self.start = start
        self.end = end
        self.jumpable_walls = self.get_jumpable_walls(walls)

    def reset(self):
        for i in range(self.maze.shape[0]):
            for j in range(self.maze.shape[1]):
                if self.maze[i][j] == 0:
                    continue
                self.maze[i][j].visited = False
                self.maze[i][j].distance = float('inf') if self.maze[i][j] != self.start else 0


    def tally_cheats(self, min_steps_saved: int):
        # for each jumpable wall, remove it and run djikstra
        # if the distance is steps_saved less than vanilla, it's a cheat
        cheatlist = []

        vanilla_dist = self.djikstra()

        for n, wall in enumerate(self.jumpable_walls, 1):
            self.reset()
            fake_wall = Node(wall)
            self.maze[wall[0]][wall[1]] = fake_wall
            cheat_dist = self.djikstra()
            if cheat_dist == -1:
                continue
            if vanilla_dist - cheat_dist >= min_steps_saved:
                cheatlist.append({"wall": wall, "steps_saved": vanilla_dist - cheat_dist})
            self.maze[wall[0]][wall[1]] = 0
            print(f"{len(cheatlist)} cheats found | {n/len(self.jumpable_walls) * 100:.2f}%", end='\r')

        print()
        return cheatlist

    def djikstra(self):
        self.set_neighbors()
        queue: list[Node] = []
        heapq.heappush(queue, (0, self.maze[self.start.pos[0]][self.start.pos[1]]))

        while len(queue) > 0:
            dist, node = heapq.heappop(queue)
            if node.visited:
                continue

            node.visited = True
            if node.pos == self.end.pos:
                return dist

            for neighbor in node.neighbors:
                if not neighbor.visited:
                    heapq.heappush(queue, (dist + 1, neighbor))
            
        return -1
    
    def set_neighbors(self):
        for i in range(self.maze.shape[0]):
            for j in range(self.maze.shape[1]):
                if self.maze[i][j] == 0:
                    continue

                self.maze[i][j].neighbors = self.get_legal_moves(self.maze[i][j])

    def get_legal_moves(self, node: Node) -> list[Node]:
        # can move up down left right
        n_moves = []
        for move in Node.moves:
            new_pos = node + move
            if not out_of_bounds(new_pos[0], new_pos[1], self.maze):
                n_node = self.maze[new_pos[0]][new_pos[1]]
                if n_node != 0:
                    n_moves.append(self.maze[new_pos[0]][new_pos[1]])

        return n_moves
                
    def get_jumpable_walls(self, walls: list[tuple[int, int]]) -> list[tuple[int, int]]:
        # if wall is surrounded by walls, remove it
        valid_coords = []
        for wall in walls:
            for move in Node.moves:
                jump = wall[0] + move[0], wall[1] + move[1]
                if out_of_bounds(jump[0], jump[1], self.maze):
                    continue

                if self.maze[jump[0]][jump[1]] == 0:
                    continue
                
                # at this point we know we can jump from the wall to stable ground
                # but is there stable ground behind us?
                opposite_move = (-move[0], -move[1])
                prev = wall[0] + opposite_move[0], wall[1] + opposite_move[1]
                if out_of_bounds(prev[0], prev[1], self.maze):
                    continue # out of bounds
                
                if self.maze[prev[0]][prev[1]] == 0:
                    continue # more wall

                # it's jumpable
                valid_coords.append(wall)
                break
        
        return valid_coords

    def plot(self, highlight_coord=None):
        m = [[0 if self.maze[i][j] == 0 else 1 for j in range(len(self.maze))] for i in range(len(self.maze[0]))]
        for wall in self.jumpable_walls:
            plt.scatter(wall[1], wall[0], c='r', marker='o', s=40)
        plt.imshow(m)
        plt.scatter(self.start.pos[1], self.start.pos[0], c='r', marker='4', s=60)
        plt.scatter(self.end.pos[1], self.end.pos[0], c='g', marker='8', s=60)
        if highlight_coord:
            plt.scatter(highlight_coord[1], highlight_coord[0], c='y', marker='x', s=120)

        plt.show()
            
    def __str__(self):
        return "\n".join(["".join(["#" if c == 0 else "S" if c == self.start else "E" if c == self.end else "." for c in row]) for row in self.maze])
    
    def __repr__(self):
        return self.__str__()

In [31]:
# reuse maze and djikstra from previous problem
def simulate(filename: str):
    maze = []
    wall_coords = []
    start = None
    end = None
    with open(filename) as f:
        i = 0
        for line in f.readlines():
            row = []
            j = 0
            for c in line.strip():
                if c == '#':
                    row.append(0)
                    wall_coords.append((i, j))
                elif c == '.':
                    row.append(Node((i, j)))
                elif c == 'S':
                    start = Node((i, j))
                    start.distance = 0
                    row.append(start)
                elif c == 'E':
                    end = Node((i, j))
                    row.append(end)
                j += 1
            maze.append(np.array(row))
            i += 1

    return Maze(np.array(maze), start, end, wall_coords)

maze = simulate('input.txt')
# maze.plot()
cheatlist = maze.tally_cheats(100)
print(f"{len(cheatlist)} cheats found")

1404 cheats found | 100.00%
1404 cheats found


In [32]:
# order by steps_saved
sorted_cheats = sorted(cheatlist, key=lambda x: x['steps_saved'], reverse=True)

# group by steps_saved
grouped_cheats = {}
for cheat in sorted_cheats:
    if cheat['steps_saved'] not in grouped_cheats:
        grouped_cheats[cheat['steps_saved']] = []
    grouped_cheats[cheat['steps_saved']].append(cheat)

for steps_saved, cheats in reversed(grouped_cheats.items()):
    print(f"There are {len(cheats)} cheats that save {steps_saved} picoseconds")

There are 22 cheats that save 100 picoseconds
There are 8 cheats that save 102 picoseconds
There are 22 cheats that save 104 picoseconds
There are 9 cheats that save 106 picoseconds
There are 23 cheats that save 108 picoseconds
There are 9 cheats that save 110 picoseconds
There are 27 cheats that save 112 picoseconds
There are 14 cheats that save 114 picoseconds
There are 25 cheats that save 116 picoseconds
There are 8 cheats that save 118 picoseconds
There are 18 cheats that save 120 picoseconds
There are 8 cheats that save 122 picoseconds
There are 18 cheats that save 124 picoseconds
There are 7 cheats that save 126 picoseconds
There are 17 cheats that save 128 picoseconds
There are 10 cheats that save 130 picoseconds
There are 19 cheats that save 132 picoseconds
There are 7 cheats that save 134 picoseconds
There are 11 cheats that save 136 picoseconds
There are 5 cheats that save 138 picoseconds
There are 14 cheats that save 140 picoseconds
There are 6 cheats that save 142 picosecon